In [2]:
import numpy as np
import pandas as pd

Importamos un paquete *util* con las funciones que realizan los cálculos, para mantener un notebook limpio.
Este paquete **no es externo**, sólo recoge código

In [ ]:
import src.miax_util as miax_util

### Carga de datos

Cargamos los datos generados en el Notebook 2

In [3]:
universe = pd.read_parquet("resources/universe.parquet")
monthly_closes = pd.read_parquet("resources/monthly_closes.parquet")
rebalancing_dates = pd.read_parquet("resources/rebalancing_dates.parquet")

print(f"monthly_closes: {monthly_closes.shape}")
print(f"eligible_universe: {universe.shape}")
print(f"rebalancing_dates: {rebalancing_dates.shape}")

monthly_closes: (103686, 4)
eligible_universe: (64510, 2)
rebalancing_dates: (133, 3)


#### NOTA
No filtramos el universo elegible por disponibilidad de precio en la fecha
de rebalanceo. Hacerlo implicaría usar información futura (saber qué activos
tendrán precio disponible ese día), es decir, lookahead.
Los activos seleccionados sin precio en el día de rebalanceo serán tratados
en el Notebook 4 como liquidez hasta el siguiente rebalanceo.

In [6]:
results = []

for _, row in rebalancing_dates.iterrows():
    rebal_date = row['date']

    # Activos elegibles para esta fecha
    eligible_symbols = universe[universe['rebal_date'] == rebal_date]['symbol'].tolist()

    # Paso A: retornos acumulados
    momentum = miax_util.calculate_momentum(rebal_date, monthly_closes, eligible_symbols)

    # Pasos B y C: z-scores, score y top 20
    top20 = miax_util.calculate_scores(momentum)
    top20 = top20.reset_index()
    top20['rebal_date'] = rebal_date
    top20['weight'] = 0.05  # 5% cada activo

    results.append(top20)

# Concatenamos todos los resultados
portfolio = pd.concat(results, ignore_index=True)

print(f"Shape: {portfolio.shape}")
print(f"Fechas de rebalanceo: {portfolio['rebal_date'].nunique()}")
portfolio.head()

Shape: (2660, 8)
Fechas de rebalanceo: 133


,symbol,R_12,R_6,Z_12,Z_6,score,rebal_date,weight
0,LUV,0.816718,0.475648,3.498600,2.277408,2.888004,2015-01-30,0.05
1,EW,0.661163,0.450260,2.721030,2.136125,2.428577,2015-01-30,0.05
2,EA,0.717584,0.291411,3.003063,1.252168,2.127616,2015-01-30,0.05
3,VRTX,0.469330,0.497171,1.762118,2.397175,2.079647,2015-01-30,0.05
4,MNST,0.469247,0.445768,1.761702,2.111131,1.936416,2015-01-30,0.05


In [7]:
# ------------------------------------------------------------
# 5. GUARDAMOS EL CSV DE SALIDA
# ------------------------------------------------------------

portfolio.to_csv("resources/portfolio_selections.csv", index=False)
print("✅ portfolio_selections.csv guardado")
print(f"Total rebalanceos: {portfolio['rebal_date'].nunique()}")
print(f"Total filas: {len(portfolio)}")

✅ portfolio_selections.csv guardado
Total rebalanceos: 133
Total filas: 2660
